# To Convert nsidc0046 snow frequency maps to climatologies

Took 25 km v04 nsidc0046 frequency data and set all pixels where snow was ever observed to 1 for the climatology mask.

These are EASE2_N25km data.  Resampled to 12.5, 6.25 and 3.125 arrays and saved all arrays as flat binary files. 

Output moved to nusnow:/share/data/pmalgos/snow_clims.

November 19, 2018
MJB


In [ ]:
%pylab notebook
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import re

In [ ]:
%cd /Users/brodzik/nsidc0046_v04/frequency
list = glob.glob("*snow.frq*bin")
list

In [ ]:
def climfile(freqfile):
    
    # Parse the frequency filename to make a clim filename
    m = re.search(r'EASE2_N25km\.snow\.frq\.(\d{2})\.(.+)', freqfile)
    mm = m.group(1)
    suffix = m.group(2)
    
    return("snow_climatology.%s.%s" % (mm, suffix))

In [ ]:
climfile(list[1])

In [ ]:
fig, ax = plt.subplots(2,3, figsize=(12,8))


In [ ]:
def doubledata(data):
    
    # repeat each column and row in each dimension
    return (np.repeat(np.repeat(data,2,axis=0),2,axis=1))


In [ ]:
def saveclimfile(data, filename):
    
    f = open(filename, "w")
    data.tofile(f)
    f.close()
    print("Saved bin data to: %s" % filename)

In [ ]:
def frq2clim(filename, fig, ax):
    
    rows = 720
    cols = 720
    
    f = open(filename, "r")
    data = np.fromfile(f, dtype=np.uint8)
    data = reshape(data, (rows, cols))

    print("Filename = %s" % filename)
    print("Dims = %s" % (str(data.shape)))
    print("Min = %.2f, Max = %.2f" % (np.min(data), np.max(data)))
    
    f.close()
    
    clim25 = np.zeros((rows,cols), dtype=np.uint8)
    snowmask = data > 0
    clim25[snowmask] = 1
    
    clim12 = doubledata(clim25)
    clim6 = doubledata(clim12)
    clim3 = doubledata(clim6)
    
    print("12.5  Dims = %s" % (str(clim12.shape)))
    print("6.25  Dims = %s" % (str(clim6.shape)))
    print("3.125 Dims = %s" % (str(clim3.shape)))
    
    ax[0,0].set_title("Frequency")
    ax[0,0].imshow(data, cmap="Blues_r", vmin=0., vmax=100., interpolation='nearest')
    ax[0,0].axis('off')
    
    ax[0,1].set_title("Climatology (25km)")
    ax[0,1].imshow(clim25, cmap="Blues_r", vmin=0., vmax=1., interpolation='nearest')
    ax[0,1].axis('off')
    
    ax[0,2].set_title("Climatology (12.5km)")
    ax[0,2].imshow(clim12, cmap="Blues_r", vmin=0., vmax=1., interpolation='nearest')
    ax[0,2].axis('off')
    
    ax[1,1].set_title("Climatology (6.25km)")
    ax[1,1].imshow(clim6, cmap="Blues_r", vmin=0., vmax=1., interpolation='nearest')
    ax[1,1].axis('off')
    
    ax[1,2].set_title("Climatology (3.125km)")
    ax[1,2].imshow(clim3, cmap="Blues_r", vmin=0., vmax=1., interpolation='nearest')
    ax[1,2].axis('off')
    
    plt.tight_layout()
    
    # Save all climfiles
    basename = climfile(filename)
    saveclimfile(clim25, "EASE2_25km.%s" % basename)
    saveclimfile(clim12, "EASE2_12.5km.%s" % basename)
    saveclimfile(clim6, "EASE2_6.25km.%s" % basename)
    saveclimfile(clim3, "EASE2_3.125km.%s" % basename)
    
    return data

In [ ]:
for f in list:
    
    print("Next file = %s" % f)
    fig, ax = plt.subplots(2,3, figsize=(12,8))
    data = frq2clim(f, fig, ax)

In [ ]:
data = frq2clim(list[0], fig, ax)